In [1]:
import sys
sys.path.append("../")
from transformers import BertModel,BertTokenizer
import pandas as pd
import utils.CommonUtils as CommonUtils
from sklearn.model_selection import train_test_split

d:\enviroment\anaconda\envs\linux_pkg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BERT_PATH = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(BERT_PATH)
bert = BertModel.from_pretrained(BERT_PATH)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
print("开始读取数据")
df_data = pd.read_csv('../output/datasource_0117_class.csv')
model_path = "bert-base-uncased" #模型路径，建议预先下载(https://huggingface.co/bert-base-chinese#)
idx_name_map, name_idx_map = CommonUtils.get_idx_name_map(df_data["name"])
num_label_map, label_num_map = CommonUtils.get_num_label_map(df_data["label"])

开始读取数据


In [4]:
import torch
import numpy as np

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.names = [name for name in df["name"]]
        self.labels = [label_num_map[label] for label in df['label']]
        self.texts = [tokenizer(text, 
                                padding='max_length',
                                max_length = 128,
                                truncation=True,
                                return_tensors="pt")
                      for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]
    
    def get_batch_names(self, idx):
        return self.names[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        batch_name = self.get_batch_names(idx)
        return batch_texts, batch_y, batch_name

In [5]:
df_train, df_test = train_test_split(df_data, test_size=0.3, random_state=666, stratify=df_data["label"])
print("train len: ", len(df_train))
print("test len: ", len(df_test))

train len:  4731
test len:  2028


In [6]:
from torch import nn

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.softmax = nn.Softmax()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.softmax(linear_output)
        return final_layer

In [7]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):
  # 通过Dataset类获取训练和验证集
    train, val = Dataset(train_data), Dataset(val_data)
    # DataLoader根据batch_size获取数据，训练时选择打乱样本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)
  # 判断是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 开始进入训练循环
    for epoch_num in range(epochs):
      # 定义两个变量，用于存储训练集的准确率和损失
            total_acc_train = 0
            total_loss_train = 0
      # 进度条函数tqdm
            for train_input, train_label, _ in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通过模型得到输出
                output = model(input_id, mask)
                # 计算损失
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                # 计算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # ------ 验证模型 -----------
            # 定义两个变量，用于存储验证集的准确率和损失
            total_acc_val = 0
            total_loss_val = 0
      # 不需要计算梯度
            with torch.no_grad():
                # 循环获取数据集，并用训练好的模型进行验证
                for val_input, val_label, _ in val_dataloader:
          # 如果有GPU，则使用GPU，接下来的操作同训练
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
  
                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'''Epochs: {epoch_num + 1} 
              | Train Loss: {total_loss_train / len(train_data): .3f} 
              | Train Accuracy: {total_acc_train / len(train_data): .3f} 
              | Val Loss: {total_loss_val / len(val_data): .3f} 
              | Val Accuracy: {total_acc_val / len(val_data): .3f}''')    

In [8]:
EPOCHS = 6
model = BertClassifier()
LR = 5e-6
train(model, df_train, df_test, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/2366 [00:00<?, ?it/s]C:\Users\kinoko_2022\AppData\Local\Temp\ipykernel_640\3557939508.py:15: UserWarning: Implicit dimension choice for 

Epochs: 1 
              | Train Loss:  0.525 
              | Train Accuracy:  0.702 
              | Val Loss:  0.482 
              | Val Accuracy:  0.779


100%|██████████| 2366/2366 [1:02:22<00:00,  1.58s/it]


Epochs: 2 
              | Train Loss:  0.452 
              | Train Accuracy:  0.843 
              | Val Loss:  0.451 
              | Val Accuracy:  0.846


100%|██████████| 2366/2366 [1:02:33<00:00,  1.59s/it]


...

Epochs: 4 
              | Train Loss:  0.416 
              | Train Accuracy:  0.912 
              | Val Loss:  0.445 
              | Val Accuracy:  0.851


100%|██████████| 2366/2366 [1:02:18<00:00,  1.58s/it]


Epochs: 5 
              | Train Loss:  0.410 
              | Train Accuracy:  0.923 
              | Val Loss:  0.440 
              | Val Accuracy:  0.866


100%|██████████| 2366/2366 [1:02:30<00:00,  1.58s/it]


Epochs: 6 
              | Train Loss:  0.405 
              | Train Accuracy:  0.934 
              | Val Loss:  0.439 
              | Val Accuracy:  0.867


In [9]:
def evaluate(model, test_data):
    
    
    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label, _ in test_dataloader:
              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)
              output = model(input_id, mask)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc   
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

C:\Users\kinoko_2022\AppData\Local\Temp\ipykernel_640\3557939508.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_layer = self.softmax(linear_output)


Test Accuracy:  0.864


In [10]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if use_cuda:
    model = model.cuda()

result_map = {}
with torch.no_grad():
    test_list = df_test.values.tolist()
    for test_data in test_list:
        name = test_data[0]
        test_input =  tokenizer(test_data[2], padding='max_length', 
                                        max_length = 128, 
                                        truncation=True,
                                        return_tensors="pt")
        mask = test_input['attention_mask'].to(device)
        input_id = test_input['input_ids'].squeeze(1).to(device)
        output = model(input_id, mask)
        result_map[name] = num_label_map[output.argmax(dim=1).item()]


C:\Users\kinoko_2022\AppData\Local\Temp\ipykernel_640\3557939508.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_layer = self.softmax(linear_output)


In [11]:
df_test["pred"] = df_test["name"].apply(lambda x: result_map[x])
df_test = df_test.loc[:,["name", "label", "pred", "summary", "description"]]
df_test.to_csv("bert_result.csv", index=False)
df_error = df_test.loc[df_test["label"] != df_test["pred"]]
df_error.to_csv("bert_result_errror.csv", index=False)

In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

report = classification_report(df_test["label"], df_test["pred"])
print(report)

matrix = confusion_matrix(df_test["label"], df_test["pred"])
print(matrix)



              precision    recall  f1-score   support

          其它       0.96      0.94      0.95       142
          工具       0.84      0.81      0.82       623
           库       0.87      0.93      0.90      1026
          服务       0.85      0.66      0.74       237

    accuracy                           0.86      2028
   macro avg       0.88      0.83      0.85      2028
weighted avg       0.86      0.86      0.86      2028

[[133   5   4   0]
 [  1 503 102  17]
 [  3  53 959  11]
 [  2  40  39 156]]
